In [1]:
from builtins import Exception
import urllib3
from bs4 import BeautifulSoup
from requests import get
import mysql.connector
import re
from datetime import date
import datetime

# Class Kompas Scraping

class dibawah merupakan proses pengambilan data dari website berita Kompas.com yang dimana pemanggilan semua fungsi dibawah tersebut nantinya akan dipanggil pada fungsi [scrapingDataKompas]

In [10]:
class Kompas:
    #deklarasi variable global
    url = ""
    mydb = ""
    keyword = ""
    urlDetailData = ""
    date = ""
    
    #fungsi dari inisialisasi data saat memanggil class pertama kali
    def __init__(self, urlBerita, pencarian,searchDate):
        self.url = urlBerita
        self.keyword = pencarian
        try:
            if(searchDate == ""):
                self.date = datetime.datetime.now()
            else:
                self.date = datetime.datetime.strptime(searchDate, '%d-%m-%Y')
        except Exception as e:
            print(e)
        self.mydb = mysql.connector.connect(host="localhost",user="root", password="",database="db-berita-mca")
         
    #fungsi menyimpan data ke database
    def simpanDatabase(self,data):
        mycursor = self.mydb.cursor()
        sql = "INSERT INTO tb_berita (media,judul_berita,label,penulis,release_date,url,content) VALUES (%s,%s,%s,%s,%s,%s,%s)"
        mycursor.execute(sql, data)
        self.mydb.commit()
        return
    
    #fungsi buat validasi data duplicate di database
    def validasiDataDatabase(self,dataBerita):
        mycursor = self.mydb.cursor()
        sql = "SELECT * FROM tb_berita WHERE url = %s"
        dataUrl = dataBerita
        url = (dataBerita,)
        mycursor.execute(sql, url)
        myresult = mycursor.fetchall()
        return myresult
            
    def getUrlDetail(self, page):
        self.urlDetailData = []
        for page in range(page):
            try:
                if(self.keyword == ""):
    #                 dateNow = self.today.strftime("%Y-%m-%d")
                    urlGet = ("https://indeks.kompas.com/?site=all&date="+self.date.strftime("%Y-%m-%d")+"&page={}".format(page+1))
                    try:
                        soup = BeautifulSoup(get(urlGet).text, 'lxml')
                        getContent = soup.find('div','latest--indeks mt2 clearfix')
                        contents = getContent.find_all('div','article__list clearfix')
                        for x in contents: 
                            news_url = x.find('a', href=True).get('href')
                            mergeUrl = (news_url+"?page=all")
                            self.urlDetailData.append(mergeUrl)
                    except Exception as e:
                        continue
                else:
#                     dateNow = self.today.strftime("%Y-%m-%d")
                    try:
                        urlGet = ("https://indeks.kompas.com/?site=all&date="+self.date.strftime("%Y-%m-%d")+"&page={}".format(page+1))
                        soup = BeautifulSoup(get(urlGet).text, 'lxml')
                        getContent = soup.find('div','latest--indeks mt2 clearfix')
                        contents = getContent.find_all('div','article__list clearfix')
                        for x in contents: 
                            news_url = x.find('a', href=True).get('href')
                            mergeUrl = (news_url+"?page=all")
                            self.urlDetailData.append(mergeUrl)
                    except Exception as e:
                        continue
            except Exception as e:
                print("format tanggal anda masukan salah")
                break
                
    def scrapingDataKompas(self):
        self.getUrlDetail(3)
        for dataBerita in self.urlDetailData:
            myresult = self.validasiDataDatabase(dataBerita)
            if(len(myresult)==0):
                try:
                    media = "KOMPAS.com"
                    web_data = get(dataBerita)
                    soup = BeautifulSoup(web_data.text, 'html.parser')
                    page = soup.find('div','container clearfix')
                    title = page.find('h1','read__title').text
                    
                    contentLabel = page.find_all('li','breadcrumb__item')
                    for x in contentLabel:
                        label = x.find('a').text
                    getAuthor = page.find('div','read__credit clearfix')
                    author = getAuthor.find('a').text                    
                    date = page.find('div','read__time').text
                    release_date = date.replace("Kompas.com - ","")
                    detail_text_class = page.find('div','read__content')
                    texts = detail_text_class.find_all('p')
                    news_content = ' '.join([text.text for text in texts])
                    data = (media,title,label,author,release_date,dataBerita,news_content)
                    self.simpanDatabase(data)
                except Exception as e:
                    continue 
            else:
                continue

In [12]:
url = 'https://www.kompas.com'
pencarian = 'vaksin'
tanggal = '15-10-2021'
testing = Kompas(url,pencarian,tanggal)
testing.getUrlDetail(4) 
testing.urlDetailData
# testing.date.strftime("%Y-%m-%d")

['https://bola.kompas.com/read/2021/10/15/23590028/5-amunisi-baru-persiraja-dari-pon-xx-papua-2021?page=all',
 'https://bola.kompas.com/read/2021/10/15/23510018/pubg-mobile-hadirkan-kembali-peta-bersalju-vikendi?page=all',
 'https://www.kompas.com/sports/read/2021/10/15/23430048/2022-upaya-penyelenggaraan-f1-tak-bentrok-dengan-piala-dunia-qatar?page=all',
 'https://regional.kompas.com/read/2021/10/15/233140178/gibran-sebut-pernyataan-fx-rudy-tenangkan-kisruh-banteng-vs-celeng-di-pdi-p?page=all',
 'https://regional.kompas.com/read/2021/10/15/231917378/diduga-aniaya-seorang-pendeta-anggota-dprd-sumba-tengah-dilaporkan-ke?page=all',
 'https://www.kompas.com/sports/read/2021/10/15/23165148/5000-fasilitas-olahraga-publik-tersedia-saat-olimpiade-paris-2024?page=all',
 'https://www.kompas.com/homey/read/2021/10/15/231000876/5-tanda-tikus-bersarang-di-rumah?page=all',
 'https://megapolitan.kompas.com/read/2021/10/15/23013151/manajemen-subway-citos-diberi-teguran-tertulis-karena-timbulkan-kerum